# Install Transformers Library

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
# from transformers import AutoModel, BertTokenizerFast
from transformers import RobertaTokenizer, RobertaModel
# specify GPU
device = torch.device("cuda")

In [4]:
df = pd.read_json('/content/drive/MyDrive/sercasm/twitter/sarcasm_detection_shared_task_twitter_training.jsonl',lines=True)
df1 = pd.read_json('/content/drive/MyDrive/sercasm/twitter/sarcasm_detection_shared_task_twitter_testing.jsonl',lines=True)

df['labels'] = df['label'].apply(lambda x: ['SARCASM', 'NOT_SARCASM'].index(x))
df1['labels'] = df1['label'].apply(lambda x: ['SARCASM', 'NOT_SARCASM'].index(x))
df1.tail()

,label,context,response,id,labels
1795,NOT_SARCASM,[I have been a business customer of MWeb @USER...,@USER @USER @USER is definitely the best out t...,twitter_1796,1
1796,SARCASM,[A woman refuses to have her temperature taken...,@USER @USER Ye let her out run wild and infect...,twitter_1797,0
1797,SARCASM,[The reason big government wants @USER out is ...,"@USER @USER @USER Thanks for that , I would ha...",twitter_1798,0
1798,NOT_SARCASM,[Happy #musicmonday and #thanks for #all your ...,@USER @USER @USER Yes also #found this on #new...,twitter_1799,1
1799,NOT_SARCASM,[Not long wrapped on the amazing #January22nd ...,@USER @USER @USER you still need to send the l...,twitter_1800,1


In [5]:
import pandas as pd
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
import string
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm.auto import tqdm
!pip install demoji
!pip install contractions
import demoji
import contractions

demoji.download_codes()
nltk.download('stopwords') 
# nltk.download('wordnet')

... OK (Got response in 0.16 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
def preprocess_text(text):
    # Tokenise words while ignoring punctuation(https://www.nltk.org/_modules/nltk/tokenize/regexp.html)
    text = re.sub(r"@",'', text)
    text = re.sub(r'\b(\w+)( \1\b)+', r'\1', text)
    text = re.sub(r"USER",'@USER', text)
    # text = re.sub(r"USER",'', text)
    text = re.sub(r'<URL>','',text)

    text = demoji.replace_with_desc(text)
    text = re.sub(r':','',text)
    text = contractions.fix(text)
    # text = re.sub(r'\.+','',text)
    # tokeniser = RegexpTokenizer(r'\s+', gaps=True)
    # tokeniser = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
    # tokeniser = RegexpTokenizer(r'\w+')
    # tokens = tokeniser.tokenize(text)

    # Lowercase and lemmatise 
    # lemmatiser = WordNetLemmatizer()
    # lemmas = [lemmatiser.lemmatize(token.lower(), pos='v') for token in tokens]
    
    # Remove stopwords
    # keywords= [lemma for lemma in lemmas if lemma not in stopwords.words('english')]
    
    # stemming
    # ps=PorterStemmer()
    # keywords=[ps.stem(x) for x in keywords]
    return text

def preprocess_text2(tweet):
    """Process tweet function.
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    """
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation (optional)
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean

In [7]:
def fun(A,l):
  if len(A)<=l:
    return A
  else:
    return A[-l::1]
    # return A[-1:-l-1:-1]
def fun1(A):
  return ''.join(A)

In [8]:
l=2

# df['tweets']= df['response'] +'reply-'+ df['context'].apply(fun,args=[l]).apply(fun1)
# df1['tweets']=df1['response'] +'reply-'+ df1['context'].apply(fun,args=[l]).apply(fun1)






df['tweets']= df['context'].apply(fun,args=[l]).apply(fun1) +'reply-'+ df['response']
df1['tweets']= df1['context'].apply(fun,args=[l]).apply(fun1) +'reply-'+ df1['response']


# df['tweets']= df['context'].apply(fun,args=[l]).apply(fun1) + df['response']
# df1['tweets']= df1['context'].apply(fun,args=[l]).apply(fun1) + df1['response']

df['tweets']= df['tweets'].apply(preprocess_text) 
df1['tweets']=df1['tweets'].apply(preprocess_text)
# df['tweets']=df['tweets']+'reply-'+ df['response'].apply(preprocess_text)
# df1['tweets']=df1['tweets']+'reply-'+ df1['response'].apply(preprocess_text)

In [9]:
# check class distribution
df['labels'].value_counts(normalize = True)

1    0.5
0    0.5
Name: labels, dtype: float64

In [10]:
df['tweets']

0       A minor child deserves privacy and should be k...
1       @USER Why is he a loser ? he is just a Press S...
2       Donald J . Trump is guilty as charged . The ev...
3       Jamie Raskin tanked Doug Collins . Collins loo...
4       Man ... y ’ all gone “ both sides ” the apocal...
                              ...                        
4995    @USER Ask your delivery guy to COME BACK and s...
4996    @USER grinning squinting face also was that #h...
4997    @USER some look at silence as boredom or priva...
4998    @USER if one of the people familiar with the m...
4999    @USER You dislikes for Atiku is still this str...
Name: tweets, Length: 5000, dtype: object

# Split train dataset into train, validation and test sets

In [11]:

X_s = df['tweets'].values
y_s = df['labels'].values

Xt_s = df1['tweets'].values
yt_s = df1['labels'].values

train_text, temp_text, train_labels, temp_labels = X_s,Xt_s,y_s,yt_s

# we will use temp_text and temp_labels to create validation and test set
val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=2018, 
                                                                test_size=0.5, 
                                                                stratify=temp_labels)
val_text, test_text, val_labels, test_labels=temp_text, temp_text, temp_labels, temp_labels

# X_train, X_test, y_train, y_test = train_test_split(X_s, y_s, test_size=0.2, random_state=42)


In [12]:
# pip install -U sentence-transformers

# Import BERT Model and BERT Tokenizer

---



---



In [13]:
# import BERT-base pretrained model
# bert = AutoModel.from_pretrained('bert-base-uncased')

# # Load the BERT tokenizer
# tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
# import BERT-base pretrained model


# from transformers import RobertaTokenizer, TFRobertaModel
# bert = RobertaModel.from_pretrained('roberta-base')

# # Load the BERT tokenizer
# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

from transformers import RobertaTokenizer, TFRobertaModel
bert = RobertaModel.from_pretrained('roberta-large')

# Load the BERT tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
# sample data
text = ["this is a bert model tutorial", "we will fine-tune a bert model"]
# encode text
sent_id = tokenizer.batch_encode_plus(text, padding=True, return_token_type_ids=False)

In [15]:
# output
print(sent_id)

{'input_ids': [[0, 9226, 16, 10, 741, 2399, 1421, 35950, 2, 1, 1, 1], [0, 1694, 40, 2051, 12, 90, 4438, 10, 741, 2399, 1421, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


# Tokenization

In [16]:
# # get length of all the messages in the train set
# seq_len = [len(i.split()) for i in train_text]
# for i in train_text:
#   print(i)
# pd.Series(seq_len).hist(bins = 30)

In [17]:
max_seq_len = 190
# max_seq_len = 128

In [18]:
total=0
for i in range(5000):
  total+=len(df['tweets'][i].split(' '))
total/5000

84.3438

In [19]:
# tokenize and encode sequences in the training set
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the test set
tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


# Convert Integer Sequences to Tensors

In [20]:
# for train set
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

# for validation set
val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

# for test set
test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.tolist())

# Create DataLoaders

In [21]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#define a batch size
batch_size = 16

# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

# Freeze BERT Parameters

In [22]:
# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = True
# print(bert)

# Define Model Architecture

In [23]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      
      # dropout layer
      self.dropout = nn.Dropout(0.1)
      
      # relu activation function
      self.relu =  nn.ReLU()

      # dense layer 1
      # self.fc1 = nn.Linear(768,256)
      self.fc1 = nn.Linear(1024,256)
      # dense layer 2 (Output layer)
      self.fc2 = nn.Linear(256,2)

      #softmax activation function
      self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model  
      _, cls_hs = self.bert(sent_id,mask, return_dict=False)
      # print("dfghjjhfchgbhlvmb jfbvfddvfgbhnjhgvcvtynubtrvcebnuexcvbnbtvrcedcvbnjbnjn11111 ",cls_hs.shape)
      x = self.fc1(cls_hs)
      # print("dfghjjhfchgbhlvmb jfbvfddvfgbhnjhgvcvtynubtrvcebnuexcvbnbtvrcedcvbnjbnjn22222 ",x.shape)
      x = self.relu(x)
      # print("dfghjjhfchgbhlvmb jfbvfddvfgbhnjhgvcvtynubtrvcebnuexcvbnbtvrcedcvbnjbnjn22222 ",x.shape)
      x = self.dropout(x)
      # print("dfghjjhfchgbhlvmb jfbvfddvfgbhnjhgvcvtynubtrvcebnuexcvbnbtvrcedcvbnjbnjn22222 ",x.shape)
      # output layer
      x = self.fc2(x)
      # print("dfghjjhfchgbhlvmb jfbvfddvfgbhnjhgvcvtynubtrvcebnuexcvbnbtvrcedcvbnjbnjn22222 ",x.shape)
      # apply softmax activation
      x = self.softmax(x)
      # print("dfghjjhfchgbhlvmb jfbvfddvfgbhnjhgvcvtynubtrvcebnuexcvbnbtvrcedcvbnjbnjn22222 ",x.shape)
      return x

In [24]:
# pass the pre-trained BERT to our define architecture
model = BERT_Arch(bert)

# push the model to GPU
model = model.to(device)

In [25]:
# optimizer from hugging face transformers
from transformers import AdamW

# define the optimizer
optimizer = AdamW(model.parameters(), lr = 2e-5)

# Find Class Weights

In [26]:
from sklearn.utils.class_weight import compute_class_weight

#compute the class weights
class_wts = compute_class_weight('balanced', np.unique(train_labels), train_labels)

print(class_wts)

[1. 1.]


In [27]:
# convert class weights to tensor
weights= torch.tensor(class_wts,dtype=torch.float)
weights = weights.to(device)

# loss function
cross_entropy  = nn.NLLLoss(weight=weights) 

# number of training epochs
epochs = 10

# Fine-Tune BERT

In [28]:
# function to train the model
def train():
  
  model.train()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save model predictions
  total_preds=[]
  
  # iterate over batches
  for step,batch in enumerate(train_dataloader):
    
    # progress update after every 50 batches.
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

    # push the batch to gpu
    batch = [r.to(device) for r in batch]
 
    sent_id, mask, labels = batch

    # clear previously calculated gradients 
    model.zero_grad()        

    # get model predictions for the current batch
    preds = model(sent_id, mask)

    # compute the loss between actual and predicted values
    loss = cross_entropy(preds, labels)

    # add on to the total loss
    total_loss = total_loss + loss.item()

    # backward pass to calculate the gradients
    loss.backward()

    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # update parameters
    optimizer.step()

    # model predictions are stored on GPU. So, push it to CPU
    preds=preds.detach().cpu().numpy()

    # append the model predictions
    total_preds.append(preds)

  # compute the training loss of the epoch
  avg_loss = total_loss / len(train_dataloader)
  
  # predictions are in the form of (no. of batches, size of batch, no. of classes).
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  #returns the loss and predictions
  return avg_loss, total_preds

In [29]:
# function for evaluating the model
def evaluate():
  
  print("\nEvaluating...")
  
  # deactivate dropout layers
  model.eval()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save the model predictions
  total_preds = []

  # iterate over batches
  for step,batch in enumerate(val_dataloader):
    
    # Progress update every 50 batches.
    if step % 50 == 0 and not step == 0:
      
      # Calculate elapsed time in minutes.
      # elapsed = format_time(time.time() - t0)
            
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

    # push the batch to gpu
    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch

    # deactivate autograd
    with torch.no_grad():
      
      # model predictions
      preds = model(sent_id, mask)

      # compute the validation loss between actual and predicted values
      loss = cross_entropy(preds,labels)

      total_loss = total_loss + loss.item()

      preds = preds.detach().cpu().numpy()

      total_preds.append(preds)

  # compute the validation loss of the epoch
  avg_loss = total_loss / len(val_dataloader) 

  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  return avg_loss, total_preds

In [30]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
!nvidia-smi

Mon Jul  5 08:13:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    27W /  70W |   2436MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Start Model Training

In [76]:
# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
epochs=1
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss,_ = train()
    
    #evaluate model
    valid_loss, _ = evaluate()
    
    #save the best model
    # if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(), 'saved_weights.pt')
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
  
    # get predictions for test data
    !nvidia-smi
    # with torch.no_grad():
    0.
    #   preds = model(test_seq.to(device), test_mask.to(device))
    #   preds = preds.detach().cpu().numpy()
    # preds = np.argmax(preds, axis = 1)
    # print(classification_report(test_y, preds))
    # print(accuracy_score(test_y, preds))

    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')


 Epoch 1 / 1
  Batch    50  of    313.
  Batch   100  of    313.
  Batch   150  of    313.
  Batch   200  of    313.
  Batch   250  of    313.
  Batch   300  of    313.

Evaluating...
  Batch    50  of    113.
  Batch   100  of    113.
Mon Jul  5 09:11:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    32W /  70W |  14298MiB / 15109MiB |      0%      Default |
|          

# Load Saved Model

In [77]:
#load weights of best model
path = 'saved_weights.pt'
model.load_state_dict(torch.load(path))

<All keys matched successfully>

# Get Predictions for Test Data

In [78]:
pred=[]
with torch.no_grad():
  for i in range(0,90):
    preds = model(test_seq[i*20:i*20+20].to(device), test_mask[i*20:i*20+20].to(device))
    preds = preds.detach().cpu().numpy()
    pred.append(preds)
print(np.array(pred).shape)

(90, 20, 2)


In [79]:
pred1=np.array(pred)
# pred1

In [80]:
pred2=pred1.reshape((1800,2))
pred2.shape

(1800, 2)

In [81]:
p=pd.DataFrame(pred2)
p.to_csv('context_2.csv')

In [82]:
pred3 = np.argmax(pred2, axis = 1)
pred3 

array([1, 0, 0, ..., 0, 1, 1])

In [83]:
print(classification_report(test_y, pred3))

              precision    recall  f1-score   support

           0       0.75      0.84      0.79       900
           1       0.82      0.71      0.76       900

    accuracy                           0.78      1800
   macro avg       0.78      0.78      0.78      1800
weighted avg       0.78      0.78      0.78      1800



In [84]:
# confusion matrix
# print(pred)
pd.crosstab(test_y, pred3)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

print(accuracy_score(test_y, pred3))

0.7777777777777778


In [85]:
brcc=pd.read_csv('76.6_roberta_large_100_32_2e-5_4epoch_preprocessing+@reply+user.csv')
brcc

,Unnamed: 0,0,1
0,0,-3.325335,-0.036623
1,1,-0.068967,-2.708409
2,2,-0.061061,-2.826262
3,3,-1.519425,-0.246972
4,4,-0.016889,-4.089531
...,...,...,...
1795,1795,-5.028877,-0.006568
1796,1796,-0.984609,-0.467743
1797,1797,-0.091802,-2.433674
1798,1798,-5.437856,-0.004358


In [86]:
brc=pd.read_csv('77.44_1context+response_batch16+3epoch_180maxlen.csv')
brc

,Unnamed: 0,0,1
0,0,-6.811113,-0.001102
1,1,-0.008530,-4.768484
2,2,-0.005769,-5.158133
3,3,-6.846656,-0.001064
4,4,-0.004320,-5.446608
...,...,...,...
1795,1795,-7.074521,-0.000847
1796,1796,-4.834524,-0.007982
1797,1797,-0.005775,-5.157138
1798,1798,-7.666234,-0.000469


In [92]:
br=pd.read_csv('77.7_2context+response_batch16+4epoch_190maxlen.csv')
br

,Unnamed: 0,0,1
0,0,-1.358476,-0.297129
1,1,-0.217651,-1.631714
2,2,-0.124306,-2.146522
3,3,-2.686106,-0.070579
4,4,-0.013663,-4.299899
...,...,...,...
1795,1795,-3.504160,-0.030533
1796,1796,-0.579078,-0.821924
1797,1797,-0.356595,-1.204161
1798,1798,-4.719265,-0.008962


In [93]:
# bc=pd.read_csv('contexto.csv')
# bc

In [165]:
# bertc=np.array(bc[['0','1']])
# bertc1=np.multiply(bertc,0.1)

bertrcc=np.array(brcc[['0','1']])
bertrcc1=np.multiply(bertrcc,0.9)

bertcr=np.array(brc[['0','1']])
bertcr1=np.multiply(bertcr,0.4)

bertr=np.array(br[['0','1']])
bertr1=np.multiply(bertr,0.8)

In [166]:
# yhats = [bertc1,bertr1,bertcr1,bertrcc1]
# yhats = [bertcr1,bertrcc1,bertr1]
yhats = [bertcr1,bertrcc1,bertr1]
import numpy as np
yhats = np.array(yhats)

# sum across ensemble members
summed = np.sum(yhats, axis=0)
# argmax across classes
result = np.argmax(summed, axis=1)

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
accuracy_score(result,test_y)

0.7944444444444444